In [1]:
%pip install scikit-learn numpy matplotlib networkx pandas scipy torch


  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached numpy-1.24.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached matplotlib-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached networkx-3.0-py3-none-any.whl (2.0 MB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:0000:0100:11
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached fonttools-4.39.2-py3-none-any.whl (1.0 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached Pillow-9.4.0-cp310-

In [5]:
import torch
print(torch.version.cuda)

11.7


In [7]:
%pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# tu trzeba wpisać odpowiednią wersję CUDA, patrz: https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html

Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 3.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 2.0 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install torch-geometric

  Using cached torch_geometric-2.2.0.tar.gz (564 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.5 MB/s eta 0:00:001.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 4.9 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.5 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773285 sha256=325ad88342669bf93835f2d07a4ca546877d60f09f121a5a83efbe89d055e427
  Stored in directory: /home/piotr/.cache/pip/wheels/c8/e4/83/5e964867e23f8a61cb8c5d5b9477617b710e96e6ebf1844562
Successfully built torch-geometric
Note: you may need to restart the kernel to use updated packages.


In [13]:
from torch_geometric.datasets import TUDataset

imdb_binary_dataset = TUDataset(root = "./datasets", name = "IMDB-BINARY")
imdb_multi_dataset = TUDataset(root = "./datasets", name = "IMDB-MULTI")
reddit_binary_dataset = TUDataset(root = "./datasets", name = "REDDIT-BINARY")
reddit_multi_5k_dataset = TUDataset(root = "./datasets", name = "REDDIT-MULTI-5K")
reddit_multi_12k_dataset = TUDataset(root = "./datasets", name = "REDDIT-MULTI-12K")

# do pobrania jeszcze dataset z cząsteczkami stąd: https://ogb.stanford.edu/docs/graphprop/#pyg

Extracting datasets/IMDB-BINARY/IMDB-BINARY.zip
Processing...
Done!
Extracting datasets/IMDB-MULTI/IMDB-MULTI.zip
Processing...
Done!
Extracting datasets/REDDIT-MULTI-5K/REDDIT-MULTI-5K.zip
Processing...
Done!
Extracting datasets/REDDIT-MULTI-12K/REDDIT-MULTI-12K.zip
Processing...
Done!


In [ ]:
import argparse
import networkx as nx
import numpy as np
import pickle
import sys, os
import time

from classifier import evaluate_clf, searchclf
from graph import load_graph, function_basis, convert2nx, get_subgraphs, new_norm, save_graphs_
from hyperparameter import load_best_params_
from sklearn.preprocessing import normalize

from originalPaper.code.randomforest import classify_forest
from tunning import merge_features

In [ ]:
def run_analysis_for(dataset, uniform_flag, n_bin, norm_flag):
    bl_feat = ['1_0_deg_min', '1_0_deg_max', '1_0_deg_mean', '1_0_deg_std', 'deg']

    # hyperparameters
    # n_bin number of bins for historgram
    # norm_flag = args.norm_flag - normalize before calling function_basis versus normalize after

    # less important hyperparameter. Used for fine tunning
    # uniform_flag - unform versus log scale. True for imdb, False for reddit.
    cdf_flag = True # cdf versus pdf. True for most dataset.
    his_norm_flag = 'yes'

    graphs, labels = load_graph(dataset)
    n = len(graphs)
    graphs_ = []
    direct = os.path.join('../data/cache/', dataset, 'norm_flag_' + str(norm_flag), '')

    try:
        with open(direct + 'graphs_', 'rb') as f:
            t0 = time.time()
            graphs_ = pickle.load(f)
            print('Finish loading existing graphs. Takes %s'%(time.time() - t0))
    except IOError:
        for i in range(n):
            if i % 50 ==0: print('#'),
            gi = convert2nx(graphs[i], i)
            subgraphs = get_subgraphs(gi)
            gi_s = [function_basis(gi, ['deg'], norm_flag=norm_flag) for gi in subgraphs]
            gi_s = [g for g in gi_s if g != None]
            graphs_.append(gi_s)
        if norm_flag == 'no': graphs_ = new_norm(graphs_, bl_feat)
        save_graphs_(graphs_, dataset=dataset, norm_flag=norm_flag)

    x_original = merge_features(dataset, graphs_, bl_feat, n_bin, his_norm_flag=his_norm_flag, cdf_flag=cdf_flag, uniform_flag=uniform_flag)
    if norm_flag=='yes':
        x = normalize(x_original, axis = 1)
    else:
        x = x_original
    y = np.array(labels)
    print(x)
    print(y)
    # you may run searchclf to search the hyperparameter yourself, or load the best_hyperparameter alreay computed for you by me
    # best_params_ = searchclf(x, y, randomseed, test_size=0.1, nonlinear_flag=nonlinear_kernel, verbose=0, print_flag='on')
    best_params_ = load_best_params_(dataset)
    print(best_params_)
    evaluate_clf(x, y, best_params_, 10, n_eval=10)
    classify_forest(x, y)


In [ ]:
run_analysis_for('imdb_multi', uniform_flag=True, n_bin=100, norm_flag='no')

In [ ]:
run_analysis_for('imdb_binary', uniform_flag=True, n_bin=70, norm_flag='no')